Libraries

In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import re

import utils as lf

In [2]:
lf.get_table('2liga')

,Poz,Zespół,Mecze_rozegrane,Pkt.,Pkt.,Z,R,P,BZ,BS,+/-,Forma
0,1,Husaria Mokotów,8,1,21,7,0,1,70,20,50,"W,W,P,W,W"
1,2,Dziki Młochów,8,2,18,6,0,2,53,35,18,"W,W,W,P,W"
2,3,UEFA Mafia Ursynów,8,3,15,5,0,3,48,38,10,"W,W,W,W,W"
3,4,KSB Warszawa,8,4,15,5,0,3,68,49,19,"W,W,P,P,P"
4,5,Orzeły Stolicy,8,5,13,4,1,3,33,26,7,"P,P,P,W,W"
5,6,AFC Goodfellas,8,6,10,3,1,4,37,47,-10,"W,W,P,W,P"
6,7,Green Lantern,8,7,9,3,0,5,43,58,-15,"P,P,W,P,W"
7,8,Tylko Zwycięstwo,8,8,9,3,0,5,28,47,-19,"P,P,W,W,P"
8,9,Warszawska Ferajna,8,9,6,2,0,6,22,61,-39,"P,P,W,P,P"
9,10,Korsarze,8,10,2,1,0,7,27,48,-21,"P,P,P,P,P"


In [3]:
lf.get_matches('2liga', round_=7)

,Kol.,Data,Godz.,Gospodarz,,Gość,Video,,Sędzia,Boisko,Raport
0,7,2023-11-12,09:00,Dziki Młochów,7 : 4,Tylko Zwycięstwo,,,Volodymyr Molokov,Arena AWF,Raport
1,7,2023-11-12,14:00,Orzeły Stolicy,3 : 5,Husaria Mokotów,,,Volodymyr Molokov,Arena AWF,Raport
2,7,2023-11-12,17:00,Korsarze,3 : 8,AFC Goodfellas,,,Volodymyr Molokov,Arena AWF,Raport
3,7,2023-11-12,19:00,UEFA Mafia Ursynów,8 : 2,Warszawska Ferajna,,,Piotr Krajczyński,Arena AWF,Raport
4,7,2023-11-12,19:00,Green Lantern,6 : 18,KSB Warszawa,,,Rafał Szczytniewski,Arena AWF,Raport


In [4]:
lf.get_squad_details('8liga', 'Kozice Warszawa')

,Imie i nazwisko,Numer,Liczba wystepów,Liczba bramek,Asysty,Kanadyjcztk,Superstar,Top6,MVP,Czerwone kartki,Zółte kartki,Stracone bramki,Samobój,Obronione karne,Czyste konto,Gold Team,ID
0,Krzysztof Wiśniewski,,8,0,0,0,2,0,0,0,0,43,0,0,0,0,7698
1,Hubert Kielak,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7697
2,Ahmad Tijjani Bala,,2,0,0,0,0,0,0,0,1,0,0,0,0,0,9815
3,Kacper Osica,,1,0,0,0,0,0,0,0,0,0,0,0,0,0,9724
4,Wojtek Soból,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7704
5,Robert Afifi,,6,2,1,3,0,1,0,0,0,0,0,0,0,0,7702
6,Patryk Cytryna,,8,2,1,3,0,0,0,0,0,0,0,0,0,0,9431
7,Krystian Kielak,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7700
8,Kamil Faryniarz,,7,1,6,7,0,0,0,0,1,0,0,0,0,0,7699
9,Kacper Napora,,5,14,6,20,8,2,1,0,0,0,0,0,0,0,7696


In [2]:
lf.get_match_details('8liga', 'Kozice Warszawa',9)

,,Imie i nazwisko,Wydarzenie,Ilość,Zespół
1,,Krzysztof Wiśniewski,Bramka stracona,7,Kozice Warszawa
3,,Alli Abdullahi,Bramka,2,Kozice Warszawa
4,,Kuba Wieteska,Bramka,2,Kozice Warszawa
5,,Patryk Cytryna,Bramka,1,Kozice Warszawa
6,,Valentine Chekwube,Bramka,2,Kozice Warszawa
8,,Alli Abdullahi,Asysta,1,Kozice Warszawa
9,,Debal Bose,Asysta,1,Kozice Warszawa
10,,Kamil Faryniarz,Asysta,2,Kozice Warszawa
11,,Robert Afifi,Asysta,1,Kozice Warszawa
13,,Alli Abdullahi,SuperStar,2,Kozice Warszawa


In [2]:
lf.get_match_details(league='8liga', team='Kozice WarSzawa', opponent='SHOT dj')

,,Imie i nazwisko,Wydarzenie,Ilość,Zespół
1,,Krzysztof Wiśniewski,Bramka stracona,5,Kozice Warszawa
3,,Alli Abdullahi,Bramka,1,Kozice Warszawa
4,,Patryk Cytryna,Bramka,1,Kozice Warszawa
5,,Robert Afifi,Bramka,2,Kozice Warszawa
6,,Valentine Chekwube,Bramka,1,Kozice Warszawa
8,,Alli Abdullahi,Asysta,1,Kozice Warszawa
9,,Kuba Wieteska,Asysta,1,Kozice Warszawa
10,,Valentine Chekwube,Asysta,2,Kozice Warszawa
12,,Krzysztof Wiśniewski,SuperStar,2,Kozice Warszawa
14,,Robert Afifi,6 - kolejki,1,Kozice Warszawa
